In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import pathlib
from tqdm import tqdm
from queue import Empty
from datetime import datetime


Quello che viene fatto è fare scraping attraverso https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022 nella stagione 2021-2022 e vengono estratte tutte le informazioni relative alle varie squadre che giocano nella stagione. 

I dati provenienti da diversi link vengono mergiati. 
Per poter classificare ho pensato a 3 classi: V (vittoria), P (persa), N (pareggio).
Il problema è che scaricando tutte le partite per tutte le squadre, ci saranno partite doppie, ad esempio: 
- Atalanta V Bologna
- Bologna P Atalanta

Per questo vado a considerare nella classificazione solo le squadre in casa, così quando si vuole predire il risultato di una partita, ottengo la classificazione per la squadra di casa.

# Dataset Serie A matches
Per la realizzazione del dataset ho trovato in letteratura l'elenco dei dati più importanti per la predizione dei risultati delle partite. 

https://iopscience.iop.org/article/10.1088/1757-899X/226/1/012099/pdf

Per questo dataset ho utilizzato le statistiche della stagione 2021/22

In [2]:
headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022"
pageTree = requests.get(page)
soup = BeautifulSoup(pageTree.text, features="lxml")

In [3]:
table = soup.select('table.stats_table')[0]
a_hrefs = table.find_all('a')

Il dizionario ha come chiave il campo estratto dalle colonne nelle tabelle dei vari link, mentre come valore la stringa con la quale deve essere sostituito.

In [4]:
rename_fields = {
    'Squadra': 'team',
    'Avversario': 'avv',
    'Rf': 'goals',
    'Rs': 'gs',
    'Tiri': 'total_shots',
    'Tiri.1': 'shots_on_target',
    'Rigori': 'goals_on_penalty',
    'Rig T': 'total_penalties',
    'Compl': 'completed_passings',
    'Tent': 'total_passings',
    'Angoli': 'corners',
    'Poss.': 'percentage_possession',
    'Falli': 'fouls',
    'Amm.': 'yellow_cards',
    'Esp.': 'red_cards',
}

In [5]:
teams = [a_href for a_href in a_hrefs if '/squadre/' in str(a_href)]        
#for team in teams:
teams.sort(key=lambda x: x.contents[0]) #ordinamento per titolo

## Estrazione dataset

In questa sezione vado ad estrarre il dataset completo di tutte le partite giocate per ogni squadra

Il dizionario sottostante contiene come chiave i link ai bottoni che permettono di portare ai link dei vari dataset della squadra e come valore il nome della parte di html che permette di accedere al dataset

In [6]:
util_hrefs = {
    'all_comps/shooting/': {
        'section': 'Tiri',
        'columns': ['Data', 'Tiri','Tiri.1','Rigori','Rig T']
        },
    'all_comps/passing/': {
        'section': 'Passaggi',
        'columns': ['Data', 'Compl.', 'Tent,']
        },
        
    'all_comps/passing_types': {
        'section': 'Tipologie di passaggi',
        'columns': ['Data', 'Angoli']
        },
    'all_comps/possession': {
        'section': 'Possesso palla',
        'columns': ['Data', 'Poss.']
        },
    'all_comps/misc': {
        'section': 'Statistiche varie',
        'columns': ['Data', 'Falli', 'Amm.', 'Esp.']
        },
}

In [7]:
datasets = pd.DataFrame()
home_games = pd.DataFrame()
away_games = pd.DataFrame()

In [8]:
for team in tqdm(teams):
    team_name = team.contents[0]
    link = f"https://fbref.com{team.get('href')}"
    data = requests.get(link)
    games = pd.read_html(data.text, match="Punteggi e partite")[0]

    matches = games[games['Competizione'] == 'Serie A']
    matches = matches[['Data', 'Stadio', 'Risultato', 'Rf','Avversario','Rs']]
    #match[home_main_factors[0]] = team_name
    matches.insert(0, "Squadra", team_name)
    #matches = matches.rename(columns={'Rf': home_main_factors[1], 'Avversario': away_main_factors[0], 'Rs': away_main_factors[1]})
    
    #ottengo il dataset della singola squadra con tutti i dati che servono
    soup = BeautifulSoup(data.text, features="lxml")
    links = soup.find_all('a')
    href_links = [l.get("href") for l in links]

    old_dataset = matches
    for href_key, section_value in util_hrefs.items():
        div_links = [l for l in href_links if l and href_key in l]
        #prendo il link e ottengo l'html
        html = requests.get(f"https://fbref.com{div_links[0]}")
        
        section, columns = section_value.items()
        section, columns = section[1], columns[1]
        #ottengo il dataset della pima sezione indicata nel match
        dataset_section = pd.read_html(html.text, match=section)[0]
        #elimino l'intestazione "Di: NomeSquadra"
        dataset_section.columns = dataset_section.columns.droplevel()
        #filtro il dataset per la competizione
        dataset_section = dataset_section[dataset_section['Competizione'] == 'Serie A']
        #filtro il dataset per le colonne selezionate
        dataset_section = dataset_section[[col for col in columns]]
        #print(dataset_section.columns)
        if section == 'Passaggi':
            column_names = dataset_section.columns.values
            column_names[1] = 'Compl'
            column_names[5] = 'Tent'
            dataset_section = dataset_section[['Data','Compl','Tent']]

        old_dataset = pd.merge(old_dataset, dataset_section, on = ["Data"])
        
    datasets = datasets.append(old_dataset)

100%|██████████| 20/20 [00:47<00:00,  2.35s/it]


Creo i due dataset, quello con le partite di casa e quello con quelle in trasferta

In [9]:
home_games = datasets[datasets['Stadio']=='Casa']
away_games = datasets[datasets['Stadio']=='Ospiti']

print(len(datasets), len(home_games), len(away_games))

760 380 380


In [10]:
rename_home_fields = {key: 'h_'+value for key, value in rename_fields.items()}
rename_home = rename_home_fields
rename_home['Rs'] = 'a_goals'
rename_home['Avversario'] = 'a_team'

rename_away_fields = {key: 'a_'+value for key, value in rename_fields.items()}
rename_away = rename_away_fields
rename_away['Rs'] = 'h_goals'
rename_away['Avversario'] = 'h_team'

In [11]:
home_games = home_games.rename(columns=rename_home)
away_games = away_games.rename(columns=rename_away)

In [13]:
normalized_matches = pd.DataFrame()

Controllo dimensioni dataset, che dovrebbero essere 20*16 partite ciascuno, quindi 380

In [14]:
print(len(home_games), len(away_games))

380 380


## Match delle stesse partite
Il dataset iniziale contiene tutte le partite da tutte le squadre, quindi si passa da un dataset di n elementi ad un dataset di n/2.
Matcho ogni partita del dataset home con la partita corrispondente del dataset away

In [15]:
away_columns_needed = ["a_team", "a_total_shots", "a_shots_on_target", "a_goals_on_penalty", "a_total_penalties", "a_corners", "a_yellow_cards", "a_red_cards", "a_fouls", "a_completed_passings",  "a_total_passings", "a_percentage_possession", "Data", "h_team"]

In [16]:
for index, home_match in home_games.iterrows():
    away_match = away_games[(away_games['Data'] == home_match['Data']) & (away_games['h_team'] == home_match['h_team']) & (away_games['a_team'] == home_match['a_team'])]
    away_match_reduced = away_match[away_columns_needed]
    home_match = pd.merge(home_match.to_frame().T, away_match_reduced, on = ["Data", "h_team", "a_team"])
    normalized_matches = normalized_matches.append(home_match)

Ordiniamo le partite secondo la data in cui vengono giocate, perché se il modello voglio allenarlo con le ultime 5 partite, mi basta prendere le 5 più recenti in base all'ordine in cui sono salvate nel file csv. Una volta ordinato il dataset per Data della partita, elimino le colonne Stadio e Data tanto non mi servono per la classificazione

In [17]:
normalized_matches['Data'] = pd.to_datetime(normalized_matches['Data'], format='%d-%m-%Y')
normalized_matches = normalized_matches.sort_values(by=['Data'], ascending=True)
normalized_matches = normalized_matches.drop(columns=['Stadio', "Data"])

Una volta definito l'ordine di appariszione delle colonne, salvo il dataframe secondo questo ordine

In [ ]:
columns_needed = ['h_team', 'h_goals', 'a_team', 'a_goals', 'Risultato', 
        'h_total_shots', 'h_shots_on_target', 'a_total_shots', 'a_shots_on_target', 
        'h_goals_on_penalty', 'h_total_penalties', 'a_goals_on_penalty', 'a_total_penalties', 
        'h_completed_passings', 'h_total_passings', 'a_completed_passings', 'a_total_passings',
        'h_corners', 'a_corners', 
        'h_fouls', 'h_yellow_cards', 'h_red_cards', 'a_yellow_cards', 'a_red_cards', 'a_fouls', 
        'h_percentage_possession', 'a_percentage_possession']
        
normalized_matches = normalized_matches[columns_needed]
normalized_matches.to_csv("normalized_matches.csv")